In [1]:
from datetime import datetime
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.operators.http_operator import SimpleHttpOperator
from airflow.operators.python_operator import PythonOperator

default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2024, 1, 1),
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
}

dag = DAG(
    'fakestore_api_dag',
    default_args=default_args,
    description='A DAG to fetch data from fakestoreapi and write to CSV and TXT files',
    schedule_interval=None,
)

def write_to_csv(response, **kwargs):
    import csv
    data = response.json()
    with open('output/products.csv', 'w', newline='') as csvfile:
        fieldnames = data[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for item in data:
            writer.writerow(item)

def write_to_txt(response, **kwargs):
    with open('output/products.txt', 'w') as f:
        f.write(response.text)

def print_done():
    print("Done!")

fetch_data = SimpleHttpOperator(
    task_id='fetch_data',
    method='GET',
    http_conn_id='',
    endpoint='/products',
    xcom_push=True,
    dag=dag,
)

write_to_csv = PythonOperator(
    task_id='write_to_csv',
    python_callable=write_to_csv,
    provide_context=True,
    dag=dag,
)

write_to_txt = PythonOperator(
    task_id='write_to_txt',
    python_callable=write_to_txt,
    provide_context=True,
    dag=dag,
)

print_done = PythonOperator(
    task_id='print_done',
    python_callable=print_done,
    dag=dag,
)

fetch_data >> [write_to_csv, write_to_txt] >> print_done


C:\Users\hello\AppData\Local\Programs\Python\Python310\lib\site-packages\airflow\__init__.py:35: RuntimeWarning: Airflow currently can be run on POSIX-compliant Operating Systems. For development, it is regularly tested on fairly modern Linux Distros and recent versions of macOS. On Windows you can run it via WSL2 (Windows Subsystem for Linux 2) or via Linux Containers. The work to add Windows support is tracked via https://github.com/apache/airflow/issues/10388, but it is not a high priority.
  warnings.warn(
OSError while attempting to symlink the latest log directory


AirflowConfigException: Cannot use relative path: `sqlite:///C:\Users\hello/airflow/airflow.db` to connect to sqlite. Please use absolute path such as `sqlite:////tmp/airflow.db`.